In [81]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import integrate
from scipy import linalg as LA
import time

In [82]:
#Constants 
global nm
nm = 1e-9
L = 4*nm

global hbar 
hbar = 1.0545718e-34  #in SI Units
#hbar_mod = 6.582e-16    # in eV.s
global es
es = 8.85418782e-12
# hbar=1
# m=0.5
global m 
m = 9.10938356e-31
 

Nd = 1e18  #  Doping in m^3
e = 1.60217662e-19
Ei = 0.56
ni = 1e10  # intrinsic in m^3
k = 1.380649e-23
T = 300
kT = k*T

global aa
aa = m/(np.pi*hbar**2)
global Ef
Ef = Ei+kT*np.log(Nd/ni)

In [98]:
#Non Uniform mesh :
N = 500
x1,dx1 = np.linspace(-L,-2*nm,10,endpoint=False,retstep=True)
x2,dx2 = np.linspace(-2*nm,2*nm,480,endpoint=False,retstep=True)
x3,dx3 = np.linspace(2*nm,L,10,retstep=True)
x = np.concatenate((x1,x2,x3),axis=0)
diff = []
for i in range(len(x)-1):
    hi = x[i+1]-x[i]
    diff.append(hi)
diff.append(dx2)
diff.append(dx1)

In [93]:
#Uniform Mesh :
N=500
x,dx=np.linspace(-L,L,N,endpoint=True,retstep=True)

In [99]:
V_init = 10.0*(np.heaviside(x+1*nm,0.5)-np.heaviside(x-1*nm,0.5)) 

In [100]:
def Schrodinger_nonuniform(V):
    A = np.zeros([N,N])
    for i in range(N):
        for j in range(N):
            hi = diff[i]
            hi_1 = diff[i-1]
            hi1 = diff[i+1]
            if j == i+1:
                A[i,j] = (-(hbar**2)/2)*(2/m)*(1/(hi1*(hi+hi_1)))
            if j == i-1:
                A[i,j] = (-(hbar**2)/2)*(2/m)*(1/(hi_1*(hi+hi_1)))
            else:
                continue
    for i in range(N):
        if i == 0:
            A[i,i] = -2*A[i,i+1] + V[i]
        elif i == N-1:
            A[i,i] = -2*A[i,i-1] + V[i]
        else:
            A[i,i] = -A[i,i+1] - A[i,i-1] + V[i]
    L = np.zeros([N,N])
    for i in range(N):
        hi = diff[i]
        hi_1 = diff[i-1]
        L[i,i] = np.sqrt((hi+hi_1)/2)
    M = np.matmul(L,L)
    B = np.matmul(M,A)
    L_inv = np.linalg.inv(L)
    temp = np.matmul(L_inv,B)
    H = np.matmul(temp,L_inv)
    w1,v1 = LA.eig(H)
    index=np.argsort(w1)
    sorted_w1=np.sort(w1[index])
    sorted_v1=v1[:,index]
    for i in range(np.shape(sorted_v1)[1]):
        sorted_v1[:,i] = np.matmul(L_inv,sorted_v1[:,i])
        conjug = np.conj(sorted_v1[:,i])
        prob = conjug*sorted_v1[:,i]
        norm = np.trapz(prob,x)
        sorted_v1[:,i] = sorted_v1[:,i]/norm
    return sorted_w1,sorted_v1

In [101]:
def Schrodinger_uniform(V):
    Lap=(-2*np.diag(np.ones(N),0) + np.diag(np.ones(N-1),1) +np.diag(np.ones(N-1),-1))/(dx**2)
    Lap[0,0],Lap[0,1],Lap[1,0]=0,0,0
    Lap[N-1,N-2],Lap[N-2,N-1],Lap[N-1,N-1]=0,0,0
    
    H=-(1.0/2.0)*(hbar**2/m)*Lap+np.diag(V)

    w1,v1=LA.eig(H)

    index=np.argsort(w1)
    sorted_w1=np.sort(w1[index]) #Energy eigen value in eV
    sorted_v1=v1[:,index]
    for i in range(np.shape(sorted_v1)[1]):
        conjug = np.conj(sorted_v1[:,i])
        prob = conjug*sorted_v1[:,i]
        norm = np.trapz(prob,x)
        sorted_v1[:,i] = sorted_v1[:,i]/norm
    return sorted_w1,sorted_v1

In [60]:
def Carrier_Density(w1,v1):
    
    f = lambda Ec: 1/(1+np.exp((Ec-Ef)/kT))
    bb = np.zeros(np.shape(w1)[0])
    for idx in range(np.shape(w1)[0]):
        Ek = w1[idx].real
        tt = integrate.quad(f, Ek, np.inf)
        bb[idx] = tt[0]

    nk = aa*bb
    nx = np.zeros(np.shape(v1)[0])
    #nx_ind = np.zeros((N,j))
    for idx1 in range(np.shape(w1)[0]):
        Ek = w1[idx1].real
        for idx2 in range(np.shape(v1)[0]):
            v1_conj = np.conj(v1[idx2,idx1])
            nx[idx2] += (v1_conj*v1[idx2,idx1])*nk[idx1]
        
        
        #nx = (nx+(v1_conj*sorted_v1[:,idx1]*nk[idx1]))
    
    #nx_ind=nx_ind/(8e-7**3)
    return nx

In [46]:
def Poisson(rho):
    #x,dx = np.linspace(0,L,N,retstep=True)
    Lap = (-2*np.diag([1]*N,k=0)+np.diag([1]*(N-1),k=1)+np.diag([1]*(N-1),k=-1))/(dx**2)
    #Next modify Lap so that it is consistent with f(0) = f(L) = 0
    Lap[0,0] = -2
    Lap[0,2] = 1
    Lap[1,0] = 1

    Lap[N-1,N-2] = 1; 
    Lap[N-2,N-1] = 1; 
    Lap[N-1,N-1] = -1;
    myLap = np.matrix(Lap)
    V = (myLap.I*rho.T)/es
    return np.asarray(V).reshape(N)

In [102]:
#To compare Times
start = time.time()
w,v = Schrodinger_nonuniform(-e*V_init)
end = time.time()
print(end-start)
print((w[1]-w[0])/e)

0.6521549224853516
(0.24898722210082436+0j)
